In [1]:
from IPython.display import HTML
from IPython.display import display
import requests
import json

In [2]:
with open("WSAL/WLASL_v0.3.json", "r") as json_file: 
    data = json.load(json_file)
# Load the uploaded JSON file
# with open(uploaded_file_name, "r") as json_file:
#     data = json.load(json_file)


In [3]:
def is_video_valid(url):
    response = requests.head(url)
    return response.status_code == 200 and response.headers.get("content-type") == "video/mp4"

def get_video_url_for_word(word, used_urls):
    for item in data:
        if item["gloss"] == word:
            instances = item["instances"]
            for instance in instances:
                url = instance.get("url")
                if url and is_video_valid(url) and url not in used_urls:
                    used_urls.add(url)  # Mark this URL as used
                    return url
            break  # Stop searching once the word is found
    return None

def text_to_asl_video(text):
    words = text.split()
    video_urls = []
    used_urls = set()

    for word in words:
        video_url = get_video_url_for_word(word, used_urls)
        if video_url:
            video_urls.append(video_url)

    return video_urls



In [4]:
input_text = '''
Get medical attention right away to document your injuries.'''
video_urls = text_to_asl_video(input_text)

# Generate HTML for the video playlist
video_playlist = '<div id="video-container"></div><script>'
video_playlist += '''
var videos = %s;
var videoContainer = document.getElementById('video-container');
var currentIndex = 0;

function playNextVideo() {
  if (currentIndex < videos.length) {
    videoContainer.innerHTML = '<video width="330" height="200" controls autoplay onended="playNextVideo()"><source src="' + videos[currentIndex] + '" type="video/mp4"></video>';
    currentIndex++;
  }
}

playNextVideo();
''' % video_urls
video_playlist += '</script>'

display(HTML(video_playlist))
